In [32]:
import pandas as pd
import chardet
import plotly_express as px

dataset_path_wfh1 = 'Data/Werkzame_beroepsbevolking__thuiswerken_04102024_193141.csv'
dataset_path_wfh2 = 'Data/Werkzame_beroepsbevolking__thuiswerken_07102024_101921.csv'

df_wfh1 = pd.read_csv(dataset_path_wfh1,delimiter=";",encoding="UTF-8-SIG")
df_wfh2 = pd.read_csv(dataset_path_wfh2,delimiter=";",encoding="UTF-8-SIG")
df_mat = pd.read_csv('Data/Mobiliteitstrend__per_rit_en_motief_02102024_122208.csv',delimiter=";",encoding="UTF-8-SIG")

df_wfh2.rename(columns={'Positie in de werkkring': 'Positie werkkring'}, inplace=True)
df_wfh2.rename(columns={'Thuiswerken': 'Thuiswerker'}, inplace=True)

columns = list(df_wfh2.columns)
columns.insert(1, columns.pop(columns.index('Thuiswerker')))
df_wfh2 = df_wfh2[columns]  

df_merged = pd.concat([df_wfh1,df_wfh2])

thuiswerker_mapping = {
    'Thuiswerker die gewoonlijk thuis werkt': 'Meestal thuiswerken',
    'Thuiswerker, incidenteel op vaste dagen': 'Soms thuiswerken',
    'Thuiswerker, incidenteel geen vaste dag': 'Soms thuiswerken',
    'Geen thuiswerker': 'Niet thuiswerken'
}

df_merged['Thuiswerker'] = df_merged['Thuiswerker'].replace(thuiswerker_mapping)

df_filtered = df_merged[
    (df_merged['Positie werkkring'] == 'Totaal') &
    (df_merged['Persoonskenmerken'] == 'Totaal personen') &
    (~df_merged['Thuiswerker'].isin(["Thuiswerker","Totaal","Meestal of soms thuiswerken","Thuiswerker die incidenteel thuis werkt"]))
]


df_filtered1 = df_wfh1[(df_wfh1['Thuiswerker'] == "Thuiswerker")&(df_wfh1['Positie werkkring'] == 'Totaal')&(df_wfh1['Persoonskenmerken'] == 'Totaal personen')]
df_filtered2 = df_wfh2[(df_wfh2['Thuiswerker'] == "Meestal of soms thuiswerken")&(df_wfh2['Positie werkkring'] == 'Totaal')&(df_wfh2['Persoonskenmerken'] == 'Totaal personen')]

df_wfh_line = pd.concat([df_filtered1,df_filtered2])


px.line(df_wfh_line,
        x="Perioden",
        y="Werkzame beroepsbevolking (x 1 000)",
        title = "Number of remote working people",
        labels={"Perioden": "Periods", "Werkzame beroepsbevolking (x 1 000)": "Number of remote working people (x1 000)"})


In [33]:
df_filtered['Thuiswerker'] = df_filtered['Thuiswerker'].replace({
    'Meestal thuiswerken': 'Often working from home',
    'Soms thuiswerken': 'Sometimes working from home',
    'Niet thuiswerken': 'Not working from home'
})

C:\Users\Pelle Boucher\AppData\Local\Temp\ipykernel_31216\4292367928.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [34]:
px.bar(df_filtered,
        x="Perioden",
        y="Werkzame beroepsbevolking (x 1 000)",
        color="Thuiswerker",
        title = "Types of remote workers",
        labels={"Perioden": "Periods", "Werkzame beroepsbevolking (x 1 000)": "Number of working people (x1 000)", "Thuiswerker": "Type of worker"},
        )




In [35]:
df_filtered_agg = df_filtered.groupby(['Perioden', 'Thuiswerker'])['Werkzame beroepsbevolking (x 1 000)'].sum().reset_index()
df_pct = df_filtered_agg.copy()
df_pct['percentage'] = df_pct.groupby('Perioden')['Werkzame beroepsbevolking (x 1 000)'].transform(
    lambda x: (x / x.sum() * 100))

fig = px.bar(df_pct,
             x="Perioden",
             y="percentage",
             color="Thuiswerker",
             title="Types of remote workers (%)",
             labels={
                 "Perioden": "Periods",
                 "percentage": "Percentage of working people",
                 "Thuiswerker": "Type of worker"
             },
             category_orders={"Thuiswerker": ["Often working from home", "Sometimes working from home", "Not working from home"]},
             text_auto='.1f'
             )

# Update layout to show percentage format
fig.update_layout(
    yaxis_title="Percentage of working people (%)",
    yaxis_range=[0, 100]
)
fig.show()